## Подготовка данных

In [ ]:
import pandas as pd
from pathlib import Path

clear_data = Path("dataset_1937770_3.txt")
fixed_data = Path("dataset.csv")

with clear_data.open("r", encoding="utf-8-sig") as fin, fixed_data.open("w", encoding="utf-8") as fout:
    for i, line in enumerate(fin, 1):
        line = line.rstrip("\n\r")
        if i == 1:
            fout.write("id,text\n")
            continue
        if "," not in line:
            raise ValueError(f"Строка {i} без запятой: {line!r}")
        left, right = line.split(",", 1)
        right_escaped = right.replace('"', '""')
        fout.write(f'{left},"{right_escaped}"\n')

task_data = pd.read_csv("dataset.csv")


In [ ]:
task_data.head()


,id,text
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку


## Обработка корпуса текста

In [ ]:
# === ЭТАП 2. Настройки и импорты (Jupyter) ===

from pathlib import Path
import re
import html
import unicodedata
import json
import time
import random
from collections import Counter
import xml.etree.ElementTree as ET  # fallback по умолчанию

# Попытаться использовать быстрый парсер lxml, если установлен:
try:
    import lxml.etree as ET  # type: ignore
    PARSER_NAME = "lxml.etree"
except Exception:
    PARSER_NAME = "xml.etree.ElementTree"

print("XML parser:", PARSER_NAME)

# Пути
DATA_DIR = Path("ruwiki_dump")
XML_PATH = DATA_DIR / "ruwiki-latest-pages-meta-current.xml"   # РАСПАКОВАННЫЙ .xml
OUT_DIR = Path("prepared_corpus")
OUT_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_PATH = OUT_DIR / "train.jsonl"
VAL_PATH = OUT_DIR / "val.jsonl"
UNIGRAM_PATH = OUT_DIR / "unigram_freq.tsv"

# Сплиты/лимиты/логи
SEED = 42
random.seed(SEED)

VAL_FRACTION = 0.05      # доля валидации
MAX_PAGES = None      # None = весь дамп; напр. 5000 для быстрого прогона
MAX_SENTENCES = 5_000_000      # None = все; напр. 100_000 для быстрого прогона
LOG_EVERY_PAGES = 100       # лог каждые N страниц
LOG_EVERY_SENTS = 5_000     # лог каждые N сохранённых предложений
STALL_SEC = 60        # watchdog: печать диагностики, если нет прогресса N секунд

# Фильтры длин
MIN_SENT_LEN = 10        # длина исходной строки (с пробелами)
MAX_SENT_LEN = 300
MIN_COMPACT_LEN = 5         # длина без пробелов
MAX_COMPACT_LEN = 180

assert XML_PATH.exists(
) and XML_PATH.suffix == ".xml", f"Нужен распакованный XML: {XML_PATH}"
print(f"XML size: {XML_PATH.stat().st_size/1e9:.2f} GB")


XML parser: xml.etree.ElementTree
XML size: 41.11 GB


In [ ]:
# === ЭТАП 2. Утилиты очистки и сегментации ===

def normalize_ws(text: str) -> str:
    text = html.unescape(text)
    text = text.replace("\u00A0", " ").replace(
        "\u202F", " ")  # NBSP/NNBSP -> space
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"[ \t\f\v]+", " ", text)
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


# Регексы для грубой очистки вики-разметки
WIKI_LINK = re.compile(r"\[\[(?:[^|\]]+\|)?([^\]]+)\]\]")
WIKI_EXT = re.compile(r"\[https?://[^\s\]]+(?:\s+([^\]]+))?\]")
TAG_REF = re.compile(r"<ref[^>/]*?/?>.*?</ref>",
                     flags=re.IGNORECASE | re.DOTALL)
TAG_REF_SELF = re.compile(r"<ref[^>]*?/?>", flags=re.IGNORECASE)
HTML_TAGS = re.compile(r"</?(nowiki|math|code|syntaxhighlight|gallery|blockquote|poem|small|sup|sub|ref|br|hr)[^>]*?>",
                       flags=re.IGNORECASE)
HEADINGS = re.compile(r"^=+[^=]+=+$", flags=re.MULTILINE)
COMMENTS = re.compile(r"<!--.*?-->", flags=re.DOTALL)
TABLES = re.compile(r"\{\|.*?\|\}", flags=re.DOTALL)
CAT_FILES = re.compile(
    r"\[\[(Категория|Category|Файл|File|Изображение|Image):[^\]]+\]\]", flags=re.IGNORECASE)
TEMPL_RE = re.compile(r"\{\{[^{}]*\}\}")


def _strip_templates_fast(text: str) -> str:
    # Ускорение: максимум 2 прохода вместо «до стабилизации»
    for _ in range(2):
        new = TEMPL_RE.sub(" ", text)
        if new == text:
            break
        text = new
    return text


def strip_wiki_markup(text: str) -> str:
    text = COMMENTS.sub(" ", text)
    text = TAG_REF.sub(" ", text)
    text = TAG_REF_SELF.sub(" ", text)
    text = HTML_TAGS.sub(" ", text)
    text = CAT_FILES.sub(" ", text)
    text = TABLES.sub(" ", text)
    # ссылку [[...|текст]] -> текст; [[текст]] -> текст
    text = WIKI_LINK.sub(lambda m: m.group(1), text)
    # внешнюю [http... текст] -> текст (или пробел)
    text = WIKI_EXT.sub(lambda m: (m.group(1) or " "), text)
    text = HEADINGS.sub(" ", text)
    text = text.replace("'''", "").replace("''", "")
    text = _strip_templates_fast(text)
    return normalize_ws(text)


# Разделение на предложения
SENT_SPLIT = re.compile(r"(?<=[\.\!\?])\s+")


def split_sentences(text: str) -> list[str]:
    parts = []
    for para in text.split("\n"):
        para = para.strip()
        if not para:
            continue
        parts.extend(SENT_SPLIT.split(para))
    out = []
    for s in parts:
        s = s.strip()
        s = re.sub(r"^[\-\*\•\·]+", "", s).strip()
        if s:
            out.append(s)
    return out

# Построение пары (слитный текст + позиции пробелов)


def to_compact_and_boundaries(s: str) -> tuple[str, list[int]]:
    chars = []
    boundaries = []
    seen_space = False
    for ch in s:
        if ch.isspace():
            seen_space = True
            continue
        if seen_space and chars:
            boundaries.append(len(chars))
        chars.append(ch)
        seen_space = False
    return "".join(chars), boundaries


def tokenize_for_unigrams(s: str) -> list[str]:
    toks = []
    for t in s.split():
        t = t.strip(".,;:!?()[]{}«»\"'`~…/\\|+=*^<>")
        if not t:
            continue
        if 1 <= len(t) <= 30 and re.fullmatch(r"[A-Za-zА-Яа-яЁё0-9\-]+", t):
            toks.append(t.lower())
    return toks


In [ ]:
# === ЭТАП 2. Namespace-агностичный итератор по страницам (ns == 0) ===

def _local(tag: str) -> str:
    """Локальное имя тега без {namespace} префикса."""
    return tag.split('}', 1)[-1] if '}' in tag else tag


def _child_text(elem, name: str) -> str:
    for ch in elem:
        if _local(ch.tag) == name:
            return ch.text or ""
    return ""


def _find_child(elem, name: str):
    for ch in elem:
        if _local(ch.tag) == name:
            return ch
    return None


def iter_wiki_pages_anyns(xml_path: Path):
    """
    Генерирует (title, text) для страниц ns==0, не завися от версии export-0.xx.
    """
    context = ET.iterparse(str(xml_path), events=("end",))
    for event, elem in context:
        if _local(elem.tag) == "page":
            if _child_text(elem, "ns") == "0":
                title = _child_text(elem, "title")
                rev = _find_child(elem, "revision")
                text = _child_text(rev, "text") if rev is not None else ""
                yield title, (text or "")
            elem.clear()


# Быстрая проверка на 2 страницы:
cnt = 0
for t, tx in iter_wiki_pages_anyns(XML_PATH):
    print("sample title:", (t or "")[:80], "| text_len:", len(tx))
    cnt += 1
    if cnt >= 2:
        break
print("iter_wiki_pages_anyns OK, first pages seen:", cnt)


sample title: Базовая статья | text_len: 32
sample title: Литва | text_len: 115278
iter_wiki_pages_anyns OK, first pages seen: 2


In [ ]:
# === ЭТАП 2. Построение корпуса с прогрессом и watchdog ===

def build_corpus_with_progress(
    xml_path: Path,
    train_path: Path,
    val_path: Path,
    unigram_path: Path,
    max_pages=MAX_PAGES,
    max_sentences=MAX_SENTENCES,
    val_fraction=VAL_FRACTION,
):
    for p in [train_path, val_path, unigram_path]:
        if p.exists():
            p.unlink()

    rng = random.Random(SEED)
    uni = Counter()
    pages = 0
    sents_seen = 0
    sents_kept = 0
    sents_filtered_len = 0
    sents_no_spaces = 0

    t0 = time.time()
    t_last = t0
    kept_last = 0
    last_title = None
    last_raw_preview = None

    with train_path.open("w", encoding="utf-8") as f_train, \
            val_path.open("w", encoding="utf-8") as f_val:

        for title, raw in iter_wiki_pages_anyns(xml_path):
            pages += 1
            last_title = title
            last_raw_preview = (raw[:400] if raw else "").replace("\n", " ")

            cleaned = strip_wiki_markup(raw)
            if cleaned:
                for s in split_sentences(cleaned):
                    sents_seen += 1

                    if not (MIN_SENT_LEN <= len(s) <= MAX_SENT_LEN):
                        sents_filtered_len += 1
                        continue

                    text_no_spaces, boundaries = to_compact_and_boundaries(s)
                    if not (MIN_COMPACT_LEN <= len(text_no_spaces) <= MAX_COMPACT_LEN):
                        sents_filtered_len += 1
                        continue
                    if len(boundaries) == 0:
                        sents_no_spaces += 1
                        continue

                    rec = {"text_no_spaces": text_no_spaces,
                           "boundaries": boundaries}
                    if rng.random() < val_fraction:
                        f_val.write(json.dumps(rec, ensure_ascii=False) + "\n")
                    else:
                        f_train.write(json.dumps(
                            rec, ensure_ascii=False) + "\n")

                    uni.update(tokenize_for_unigrams(s))
                    sents_kept += 1

                    # Логи по предложениям
                    if LOG_EVERY_SENTS and (sents_kept % LOG_EVERY_SENTS == 0):
                        now = time.time()
                        speed = (sents_kept - kept_last) / \
                            max(1e-9, (now - t_last))
                        print(f"[SENTS] kept={sents_kept:,} seen={sents_seen:,} "
                              f"filtered_len={sents_filtered_len:,} no_space={sents_no_spaces:,} "
                              f"| ~{speed:.1f} kept/s | elapsed={now - t0:.1f}s",
                              flush=True)
                        kept_last = sents_kept
                        t_last = now

                    # Watchdog: долго нет роста kept
                    if (time.time() - t_last) > STALL_SEC:
                        print("\n[WATCHDOG] Нет роста kept > "
                              f"{STALL_SEC}s. Последняя страница:\n"
                              f"  title: {last_title!r}\n"
                              f"  raw[:400]: {last_raw_preview}\n", flush=True)
                        t_last = time.time()

                    if max_sentences and sents_kept >= max_sentences:
                        break

            if LOG_EVERY_PAGES and (pages % LOG_EVERY_PAGES == 0):
                print(
                    f"[PAGES] pages={pages:,} | kept={sents_kept:,} | seen={sents_seen:,}", flush=True)

            if max_pages and pages >= max_pages:
                break
            if max_sentences and sents_kept >= max_sentences:
                break

    with unigram_path.open("w", encoding="utf-8") as f_uni:
        for w, c in uni.most_common():
            f_uni.write(f"{w}\t{c}\n")

    print("=== ИТОГО ===")
    print(f"Страниц обработано: {pages:,}")
    print(f"Предложений рассмотрено: {sents_seen:,}")
    print(f"Сохранено (train+val): {sents_kept:,}")
    print(f"Отсеяно по длине: {sents_filtered_len:,}")
    print(f"Без пробелов (нет меток): {sents_no_spaces:,}")
    print(
        f"Train строк: {sum(1 for _ in train_path.open('r', encoding='utf-8')):,}")
    print(
        f"Val   строк: {sum(1 for _ in val_path.open('r', encoding='utf-8')):,}")
    print(
        f"Уникальных униграмм: {sum(1 for _ in unigram_path.open('r', encoding='utf-8')):,}")
    print(f"Время (факт): {time.time() - t0:.1f} s")


# Запуск построения корпуса (при необходимости поставьте лимиты MAX_PAGES / MAX_SENTENCES)
build_corpus_with_progress(
    XML_PATH,
    TRAIN_PATH,
    VAL_PATH,
    UNIGRAM_PATH,
    max_pages=MAX_PAGES,
    max_sentences=MAX_SENTENCES,
    val_fraction=VAL_FRACTION,
)


[SENTS] kept=5,000 seen=7,036 filtered_len=1,955 no_space=81 | ~37104.7 kept/s | elapsed=0.1s
[SENTS] kept=10,000 seen=13,994 filtered_len=3,858 no_space=136 | ~37889.6 kept/s | elapsed=0.3s
[SENTS] kept=15,000 seen=20,269 filtered_len=5,048 no_space=221 | ~43157.1 kept/s | elapsed=0.4s
[SENTS] kept=20,000 seen=26,329 filtered_len=6,061 no_space=268 | ~46386.9 kept/s | elapsed=0.5s
[SENTS] kept=25,000 seen=32,486 filtered_len=7,145 no_space=341 | ~38096.1 kept/s | elapsed=0.6s
[PAGES] pages=100 | kept=26,223 | seen=34,009
[SENTS] kept=30,000 seen=38,654 filtered_len=8,220 no_space=434 | ~39197.5 kept/s | elapsed=0.7s
[PAGES] pages=200 | kept=33,328 | seen=42,956
[SENTS] kept=35,000 seen=45,078 filtered_len=9,558 no_space=520 | ~37805.5 kept/s | elapsed=0.9s
[PAGES] pages=300 | kept=39,547 | seen=51,016
[SENTS] kept=40,000 seen=51,574 filtered_len=10,950 no_space=624 | ~38286.5 kept/s | elapsed=1.0s
[SENTS] kept=45,000 seen=57,827 filtered_len=12,148 no_space=679 | ~38351.8 kept/s | ela

In [ ]:
# === ЭТАП 2. (Опционально) Добавление синтетики с ПЕРВЫМИ версиями списков ===
# Если не хотите синтетику сейчас — ПРОСТО НЕ ЗАПУСКАЙТЕ эту ячейку.

from collections import Counter

UNITS = ["см", "мм", "м", "км", "кг", "г", "л", "мл", "дюйм",
         "дюйма", "дюймов", "gb", "mb", "tb", "гб", "мб", "тб"]
BRANDS = ["iphone", "samsung", "xiaomi", "huawei", "sony",
          "nokia", "poco", "asus", "lenovo", "honor", "pixel", "oneplus"]
MODELS = ["14", "14pro", "14proMax", "13", "s23", "s23ultra", "s24",
          "s24plus", "mi13", "mi14", "12t", "rtx4060", "rtx4070ti", "rx7800xt"]
MISC_LEFT = ["usb", "hdmi", "typec", "usbc", "sd", "microsd",
             "ssd", "hdd", "ddr4", "ddr5", "wifi", "bluetooth"]
MISC_RIGHT = ["кабель", "переходник", "адаптер", "карта",
              "накопитель", "модуль", "память", "зарядка", "чехол", "блокпитания"]

SYNTHETIC_COUNT = 5000  # при необходимости измените


def synth_examples(n=SYNTHETIC_COUNT):
    rng = random.Random(SEED + 7)
    out = []
    # 1) число + единица измерения
    for _ in range(n // 3):
        num = str(rng.choice([1, 2, 3, 4, 5, 10, 12, 14,
                  15, 16, 24, 27, 32, 64, 128, 256, 512, 1024]))
        unit = rng.choice(UNITS)
        tail = rng.choice(
            ["", " кабель", " экран", " дисплей", " память", " вес"])
        text = f"{num} {unit}{tail}"
        out.append(text)
    # 2) бренд+модель (слитно или с хвостом)
    for _ in range(n // 3):
        b = rng.choice(BRANDS)
        m = rng.choice(MODELS)
        tail = rng.choice(["", " телефон", " смартфон", " чехол", " зарядка"])
        text = rng.choice([f"{b}{m}{tail}", f"{b} {m}{tail}"])
        out.append(text)
    # 3) <misc_left> <misc_right> <num><unit>
    for _ in range(n - 2*(n//3)):
        left = rng.choice(MISC_LEFT)
        right = rng.choice(MISC_RIGHT)
        num = str(rng.choice(
            [1, 2, 3, 5, 10, 15, 20, 25, 30, 50, 100, 150, 200]))
        unit = rng.choice(UNITS)
        text = f"{left} {right} {num}{unit}"
        out.append(text)
    rng.shuffle(out)
    return out


def append_synthetic(train_path: Path, unigram_path: Path, examples: list[str]):
    uni_counter = Counter()
    added = 0
    with train_path.open("a", encoding="utf-8") as f_train:
        for s in examples:
            s_norm = normalize_ws(s)
            # ослабим фильтры для коротких синтетических фраз
            if len(s_norm) < 5:
                continue
            text_no_spaces, boundaries = to_compact_and_boundaries(s_norm)
            if len(text_no_spaces) < 3 or len(text_no_spaces) > MAX_COMPACT_LEN:
                continue
            if len(boundaries) == 0:
                continue
            rec = {"text_no_spaces": text_no_spaces, "boundaries": boundaries}
            f_train.write(json.dumps(rec, ensure_ascii=False) + "\n")
            uni_counter.update(tokenize_for_unigrams(s_norm))
            added += 1

    existing = Counter()
    if unigram_path.exists():
        with unigram_path.open("r", encoding="utf-8") as f_uni:
            for line in f_uni:
                w, c = line.rstrip("\n").split("\t")
                existing[w] += int(c)
    existing.update(uni_counter)
    with unigram_path.open("w", encoding="utf-8") as f_uni:
        for w, c in existing.most_common():
            f_uni.write(f"{w}\t{c}\n")

    print(f"Синтетики добавлено в train: {added}")
    print(
        f"Униграммы обновлены: {sum(1 for _ in unigram_path.open('r', encoding='utf-8'))} строк")


# Генерация и добавление (опционально)
synth = synth_examples(SYNTHETIC_COUNT)
append_synthetic(TRAIN_PATH, UNIGRAM_PATH, synth)


Синтетики добавлено в train: 4738
Униграммы обновлены: 1570271 строк


In [ ]:
# === ЭТАП 2. Санитайзер JSONL и быстрый просмотр примеров ===

def sanitize_jsonl(path_str: str):
    path = Path(path_str)
    if not path.exists():
        print("Нет файла:", path)
        return
    tmp = path.with_suffix(path.suffix + ".fixed")
    bad = 0
    with path.open("r", encoding="utf-8", errors="ignore") as fin, \
            tmp.open("w", encoding="utf-8") as fout:
        for line in fin:
            s = line.rstrip("\n")
            if not s.strip():
                continue
            try:
                json.loads(s)
                fout.write(s + "\n")
            except json.JSONDecodeError:
                bad += 1
    if bad:
        path.rename(path.with_suffix(path.suffix + ".bak"))
        tmp.rename(path)
        print(
            f"[sanitize] Исправлено. Отброшено битых строк: {bad}. Бэкап: {path.with_suffix(path.suffix + '.bak').name}")
    else:
        tmp.unlink(missing_ok=True)
        print("[sanitize] Файл целый.")


def sample_jsonl(path: Path, k=5):
    items = []
    with path.open("r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            if i >= k:
                break
            items.append(json.loads(line))
    for rec in items:
        t = rec["text_no_spaces"]
        b = rec["boundaries"]
        # восстановим для проверки
        chars = list(t)
        for pos in sorted(b, reverse=True):
            if 0 <= pos <= len(chars):
                chars.insert(pos, " ")
        restored = "".join(chars)
        print("no_spaces:", t[:120])
        print("boundaries:", b[:20], ("..." if len(b) > 20 else ""))
        print("restore :", restored[:120])
        print("---")


# Санитайз и сэмпл
sanitize_jsonl(str(TRAIN_PATH))
sanitize_jsonl(str(VAL_PATH))

print("\nПримеры из train.jsonl:")
sample_jsonl(TRAIN_PATH, k=5)
print("\nПримеры из val.jsonl:")
sample_jsonl(VAL_PATH, k=3)

print("\nРазмеры файлов:")
print("train.jsonl :", TRAIN_PATH.stat(
).st_size if TRAIN_PATH.exists() else 0, "байт")
print("val.jsonl   :", VAL_PATH.stat().st_size if VAL_PATH.exists() else 0, "байт")
print("unigram_freq:", UNIGRAM_PATH.stat(
).st_size if UNIGRAM_PATH.exists() else 0, "байт")


[sanitize] Файл целый.
[sanitize] Файл целый.

Примеры из train.jsonl:
no_spaces: #REDIRECTЗаглавнаястраница
boundaries: [9, 18] 
restore : #REDIRECT Заглавная страница
---
no_spaces: Площадь—км2.
boundaries: [7, 8, 11] 
restore : Площадь — км2 .
---
no_spaces: Населениесоставляетчеловек(октябрь,2024).
boundaries: [9, 19, 26, 35, 40] 
restore : Население составляет человек (октябрь, 2024) .
---
no_spaces: 6сентября1991годаГосударственныйсоветСССРпризналнезависимостьЛитвы.
boundaries: [1, 9, 13, 17, 32, 37, 41, 48, 61] 
restore : 6 сентября 1991 года Государственный совет СССР признал независимость Литвы.
---
no_spaces: Литва—членООН(1991),ОБСЕ(1991),СоветаЕвропы(1993),ВТО(2001),Европейскогосоюза(2004),НАТО(2004)иОЭСР(2018).
boundaries: [5, 6, 10, 13, 20, 24, 31, 37, 43, 50, 53, 60, 72, 77, 84, 88, 94, 95, 99] 
restore : Литва — член ООН (1991), ОБСЕ (1991), Совета Европы (1993), ВТО (2001), Европейского союза (2004), НАТО (2004) и ОЭСР (2
---

Примеры из val.jsonl:
no_spaces: Литва́(),

## Признаки + Обучение классификатора

In [ ]:
# === ЭТАП 3. ОБУЧЕНИЕ КЛАССИФИКАТОРА ГРАНИЦ ПРОБЕЛОВ ===
from pathlib import Path
import json
import math
import random
import re
from collections import Counter
import numpy as np
from scipy import sparse

from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import SGDClassifier
from sklearn.utils import murmurhash3_32
import joblib

# Пути к артефактам Этапа 2
DATA_DIR = Path("prepared_corpus")
TRAIN_PATH = DATA_DIR / "train.jsonl"
VAL_PATH = DATA_DIR / "val.jsonl"
# можно отсутствовать — тогда лексикон не используется
UNIGRAM_PATH = DATA_DIR / "unigram_freq.tsv"

# Куда сохраняем модель
ART_DIR = Path("artifacts_boundary")
ART_DIR.mkdir(parents=True, exist_ok=True)
MODEL_PATH = ART_DIR / "boundary_sgd.joblib"
CFG_PATH = ART_DIR / "config_boundary.json"
THR_PATH = ART_DIR / "best_threshold.txt"

# Гиперпараметры признаков/модели
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

WINDOW = 5                  # контекст по 5 символов слева и справа
# размер хеш-пространства для FeatureHasher (262144)
N_FEATURES = 2**18
# доля оставляемых "0"-примеров (оставим все "1" и 30% "0")
NEGATIVE_DOWNSAMPLE = 0.3
BATCH_LINES = 5000          # сколько строк train.jsonl обрабатываем на один partial_fit
ALPHA = 1e-5                # L2-регуляризация у SGDClassifier
MAX_EPOCHS = 2              # проходов по train.jsonl (стримингово)
USE_UNIGRAM = True          # использовать ли униграммный лексикон как флаги

# Порог для «словарного» слова (если используем лексикон)
UNIGRAM_MIN_COUNT = 5

# Тюнинг порога
THRESH_GRID = np.linspace(0.1, 0.9, 33)  # сетка из 33 точек

print("train.jsonl exists:", TRAIN_PATH.exists())
print("val.jsonl   exists:", VAL_PATH.exists())
print("unigram_freq exists:", UNIGRAM_PATH.exists())


train.jsonl exists: True
val.jsonl   exists: True
unigram_freq exists: True


In [ ]:
# Подгружаем униграммный словарь: word -> bool(частотен)
LEXICON = None
if USE_UNIGRAM and UNIGRAM_PATH.exists():
    LEXICON = set()
    with UNIGRAM_PATH.open("r", encoding="utf-8") as f:
        for line in f:
            w, c = line.rstrip("\n").split("\t")
            if int(c) >= UNIGRAM_MIN_COUNT:
                LEXICON.add(w)
    print(
        f"[LEXICON] загружено слов: {len(LEXICON)} (min_count={UNIGRAM_MIN_COUNT})")
else:
    print("[LEXICON] не используется или файл отсутствует.")


[LEXICON] загружено слов: 385143 (min_count=5)


In [ ]:
# Чтение jsonl построчно (стриминг)
def iter_jsonl(path: Path):
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            yield json.loads(line)

# Категории символов (Юникод)


def is_cyr(ch: str) -> bool:
    return 'А' <= ch <= 'я' or ch in "Ёё"


def is_lat(ch: str) -> bool:
    oc = ord(ch)
    return (65 <= oc <= 90) or (97 <= oc <= 122)


def is_digit(ch: str) -> bool:
    return ch.isdigit()


def is_punct(ch: str) -> bool:
    return ch in r""".,;:!?-—()[]{}«»"'/\|+*=_~%^<>…"""


def cat(ch: str) -> str:
    if is_cyr(ch):
        return "cyr"
    if is_lat(ch):
        return "lat"
    if is_digit(ch):
        return "dig"
    if is_punct(ch):
        return "pnc"
    return "oth"

# Служебные флаги на переходах


def trans(a: str, b: str) -> str:
    return f"{a}->{b}"

# Правило «словоподобного» символа (для предполагаемых токенов)


def is_word_char(ch: str) -> bool:
    return is_cyr(ch) or is_lat(ch) or is_digit(ch) or ch in "-_"

# Быстрый поиск «локальных токенов» слева/справа от позиции


def local_tokens(text: str, pos: int) -> tuple[str, str, str]:
    """
    pos — позиция вставки пробела (между pos-1 и pos), 1..len(text)-1.
    Возвращает (left_token, right_token, merged_token) в нижнем регистре.
    """
    n = len(text)
    i = pos - 1
    j = pos
    # идём влево
    L = i
    while L >= 0 and is_word_char(text[L]):
        L -= 1
    left_tok = text[L+1:i+1]
    # вправо
    R = j
    while R < n and is_word_char(text[R]):
        R += 1
    right_tok = text[j:R]
    merged = (left_tok + right_tok) if (left_tok and right_tok) else ""
    return left_tok.lower(), right_tok.lower(), merged.lower()

# Признаки одной позиции


def boundary_features(text: str, pos: int, window: int, lexicon=None) -> dict:
    """
    text: компактная строка без пробелов
    pos:  позиция 1..len(text)-1 (между символами)
    """
    feats = {}
    n = len(text)
    Lch = text[pos-1]
    Rch = text[pos]
    Lc = cat(Lch)
    Rc = cat(Rch)

    # Базовые свойства
    feats[f"LC:{Lch}"] = 1
    feats[f"RC:{Rch}"] = 1
    feats[f"LCAT:{Lc}"] = 1
    feats[f"RCAT:{Rc}"] = 1
    feats[f"TRAN:{trans(Lc,Rc)}"] = 1

    # Регистры
    feats["L_isupper"] = 1 if Lch.isalpha() and Lch.isupper() else 0
    feats["R_isupper"] = 1 if Rch.isalpha() and Rch.isupper() else 0

    # Символьное окно
    for k in range(1, window+1):
        if pos-1-k >= 0:
            ch = text[pos-1-k]
            feats[f"L{k}:{ch}"] = 1
            feats[f"L{k}C:{cat(ch)}"] = 1
        if pos+k < n:
            ch = text[pos+k]
            feats[f"R{k}:{ch}"] = 1
            feats[f"R{k}C:{cat(ch)}"] = 1

    # Переходы цифра↔буква/кирилл↔латиница
    feats["dig_to_alpha"] = 1 if (
        is_digit(Lch) and (is_cyr(Rch) or is_lat(Rch))) else 0
    feats["alpha_to_dig"] = 1 if (
        (is_cyr(Lch) or is_lat(Lch)) and is_digit(Rch)) else 0
    feats["cyr_to_lat"] = 1 if (is_cyr(Lch) and is_lat(Rch)) else 0
    feats["lat_to_cyr"] = 1 if (is_lat(Lch) and is_cyr(Rch)) else 0
    feats["punct_left"] = 1 if is_punct(Lch) else 0
    feats["punct_right"] = 1 if is_punct(Rch) else 0

    # Лексические флаги по локальным токенам
    if lexicon is not None:
        lt, rt, merged = local_tokens(text, pos)
        feats["lex_left"] = 1 if lt and lt in lexicon else 0
        feats["lex_right"] = 1 if rt and rt in lexicon else 0
        feats["lex_merged"] = 1 if merged and merged in lexicon else 0
        # длины «слов» — полезны как численные
        feats["len_lt"] = len(lt)
        feats["len_rt"] = len(rt)
        feats["len_merged"] = len(merged)
    else:
        feats["len_lt"] = 0
        feats["len_rt"] = 0
        feats["len_merged"] = 0

    # Позиционные фичи
    feats["rel_pos"] = pos / n
    feats["is_start_near"] = 1 if pos <= 2 else 0
    feats["is_end_near"] = 1 if (n - pos) <= 2 else 0
    return feats


In [ ]:
# Преобразование партии строк в матрицу признаков и вектор меток
HASher = FeatureHasher(n_features=N_FEATURES,
                       input_type="dict", alternate_sign=False)


def build_batch_Xy(records, window=WINDOW, lexicon=LEXICON, neg_downsample=NEGATIVE_DOWNSAMPLE, seed=SEED):
    rng = random.Random(seed)
    feat_dicts = []
    labels = []
    for rec in records:
        text = rec["text_no_spaces"]
        gold = set(rec["boundaries"])  # 0-based позиции «вставить пробел»
        n = len(text)
        if n <= 1:
            continue
        # все потенциальные позиции: 1..n-1
        for pos in range(1, n):
            y = 1 if pos in gold else 0
            if y == 0:
                # даунсэмпл негатива
                if rng.random() > neg_downsample:
                    continue
            feat_dicts.append(boundary_features(text, pos, window, lexicon))
            labels.append(y)
    if not feat_dicts:
        # пустой батч
        return sparse.csr_matrix((0, N_FEATURES)), np.zeros((0,), dtype=np.int8)
    X = HASher.transform(feat_dicts)
    y = np.array(labels, dtype=np.int8)
    return X, y


In [ ]:
# SGD "логистическая регрессия" с partial_fit — обучается стримингом, не кладёт всё в память

clf = SGDClassifier(
    loss="log_loss",      # логистическая регрессия
    penalty="l2",
    alpha=ALPHA,
    random_state=SEED,
    max_iter=1,           # один проход внутри partial_fit; внешние эпохи задаём сами
    warm_start=False,
    n_jobs=-1,
)

classes = np.array([0, 1], dtype=np.int64)


def train_streaming(train_path: Path, epochs=MAX_EPOCHS, batch_lines=BATCH_LINES):
    total_seen = 0
    for ep in range(1, epochs+1):
        print(f"\n=== Эпоха {ep}/{epochs} ===")
        batch = []
        for i, rec in enumerate(iter_jsonl(train_path), 1):
            batch.append(rec)
            if len(batch) >= batch_lines:
                X, y = build_batch_Xy(batch)
                if X.shape[0] > 0:
                    clf.partial_fit(X, y, classes=classes)
                    total_seen += X.shape[0]
                print(
                    f"[train] эп={ep} | обработано строк: {i} | позиций в батче: {X.shape[0]} | всего позиций: {total_seen}")
                batch.clear()
        # хвост
        if batch:
            X, y = build_batch_Xy(batch)
            if X.shape[0] > 0:
                clf.partial_fit(X, y, classes=classes)
                total_seen += X.shape[0]
            print(
                f"[train] эп={ep} | хвост | позиций в батче: {X.shape[0]} | всего позиций: {total_seen}")
    print("\n[train] готово. Всего позиций обучено:", total_seen)


train_streaming(TRAIN_PATH, epochs=MAX_EPOCHS, batch_lines=BATCH_LINES)



=== Эпоха 1/2 ===
[train] эп=1 | обработано строк: 5000 | позиций в батче: 149892 | всего позиций: 149892
[train] эп=1 | обработано строк: 10000 | позиций в батче: 159488 | всего позиций: 309380
[train] эп=1 | обработано строк: 15000 | позиций в батче: 149266 | всего позиций: 458646
[train] эп=1 | обработано строк: 20000 | позиций в батче: 149257 | всего позиций: 607903
[train] эп=1 | обработано строк: 25000 | позиций в батче: 156821 | всего позиций: 764724
[train] эп=1 | обработано строк: 30000 | позиций в батче: 154820 | всего позиций: 919544
[train] эп=1 | обработано строк: 35000 | позиций в батче: 160389 | всего позиций: 1079933
[train] эп=1 | обработано строк: 40000 | позиций в батче: 146303 | всего позиций: 1226236
[train] эп=1 | обработано строк: 45000 | позиций в батче: 149910 | всего позиций: 1376146
[train] эп=1 | обработано строк: 50000 | позиций в батче: 120744 | всего позиций: 1496890
[train] эп=1 | обработано строк: 55000 | позиций в батче: 137285 | всего позиций: 163417

In [ ]:
from math import isfinite


def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def predict_probs_for_text(text: str):
    # Вернём proba для всех позиций 1..n-1
    n = len(text)
    if n <= 1:
        return np.zeros((0,), dtype=np.float32)
    feats = [boundary_features(text, pos, WINDOW, LEXICON)
             for pos in range(1, n)]
    X = HASher.transform(feats)
    # SGDClassifier даёт decision_function; переведём в proba через сигмоиду
    dec = clf.decision_function(X)
    probs = sigmoid(dec).astype(np.float32)
    return probs  # длины n-1, индекс 0 соответствует позиции 1 в строке


def f1_per_string(gold_set, pred_set):
    # gold_set, pred_set — множества позиций (целые индексы)
    if not gold_set and not pred_set:
        return 1.0
    if not gold_set and pred_set:
        return 0.0
    if gold_set and not pred_set:
        return 0.0
    tp = len(gold_set & pred_set)
    fp = len(pred_set - gold_set)
    fn = len(gold_set - pred_set)
    if tp == 0:
        return 0.0
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    return 2 * prec * rec / (prec + rec)


def evaluate_on_val(val_path: Path, thresholds=THRESH_GRID):
    # Для подбора порога посчитаем proba для каждой строки один раз
    probs_list = []   # список np.array (длина = n-1)
    golds_list = []   # список set(int)
    lengths = []      # длина строки (n)
    cnt = 0
    for rec in iter_jsonl(val_path):
        text = rec["text_no_spaces"]
        gold = set(rec["boundaries"])
        p = predict_probs_for_text(text)
        probs_list.append(p)
        golds_list.append(gold)
        lengths.append(len(text))
        cnt += 1
        if cnt % 2000 == 0:
            print(f"[val] подготовлено строк: {cnt}")

    # Подбор порога
    best_thr = 0.5
    best_f1 = -1.0
    grid_scores = []
    for thr in thresholds:
        f1s = []
        for p, gold, n in zip(probs_list, golds_list, lengths):
            if n <= 1:
                f1s.append(1.0 if len(gold) == 0 else 0.0)
                continue
            # позиции 1..n-1, а p[0] соответствует позиции 1
            pred_pos = set((np.where(p >= thr)[0] + 1).tolist())
            f1s.append(f1_per_string(gold, pred_pos))
        mean_f1 = float(np.mean(f1s)) if f1s else 0.0
        grid_scores.append((thr, mean_f1))
        if mean_f1 > best_f1:
            best_f1 = mean_f1
            best_thr = thr
    print("\n[val] Порог → F1:")
    for thr, sc in grid_scores[:10]:
        print(f"  thr={thr:.3f}  F1={sc*100:.2f}")
    print("  ...")
    print(f"[val] ЛУЧШИЙ ПОРОГ: {best_thr:.3f} | F1={best_f1*100:.2f}")

    return best_thr, best_f1, grid_scores


best_thr, best_f1, grid = evaluate_on_val(VAL_PATH, thresholds=THRESH_GRID)


[val] подготовлено строк: 2000
[val] подготовлено строк: 4000
[val] подготовлено строк: 6000
[val] подготовлено строк: 8000
[val] подготовлено строк: 10000
[val] подготовлено строк: 12000
[val] подготовлено строк: 14000
[val] подготовлено строк: 16000
[val] подготовлено строк: 18000
[val] подготовлено строк: 20000
[val] подготовлено строк: 22000
[val] подготовлено строк: 24000
[val] подготовлено строк: 26000
[val] подготовлено строк: 28000
[val] подготовлено строк: 30000
[val] подготовлено строк: 32000
[val] подготовлено строк: 34000
[val] подготовлено строк: 36000
[val] подготовлено строк: 38000
[val] подготовлено строк: 40000
[val] подготовлено строк: 42000
[val] подготовлено строк: 44000
[val] подготовлено строк: 46000
[val] подготовлено строк: 48000
[val] подготовлено строк: 50000
[val] подготовлено строк: 52000
[val] подготовлено строк: 54000
[val] подготовлено строк: 56000
[val] подготовлено строк: 58000
[val] подготовлено строк: 60000
[val] подготовлено строк: 62000
[val] подгот

In [ ]:
import json

# Сохраняем модель и конфиг
joblib.dump(clf, MODEL_PATH)
with CFG_PATH.open("w", encoding="utf-8") as f:
    json.dump({
        "window": WINDOW,
        "n_features": N_FEATURES,
        "neg_downsample": NEGATIVE_DOWNSAMPLE,
        "alpha": ALPHA,
        "use_unigram": bool(LEXICON is not None),
        "unigram_min_count": UNIGRAM_MIN_COUNT if LEXICON is not None else None,
        "seed": SEED
    }, f, ensure_ascii=False, indent=2)

with THR_PATH.open("w", encoding="utf-8") as f:
    f.write(f"{best_thr:.6f}\n")

print("Сохранено:")
print("  Модель:", MODEL_PATH.resolve())
print("  Конфиг:", CFG_PATH.resolve())
print("  Порог :", THR_PATH.resolve())

# Быстрая sanity-проверка на нескольких строках из вал


def restore_with_positions(text_no_spaces: str, positions: set[int]) -> str:
    chars = list(text_no_spaces)
    for pos in sorted(positions, reverse=True):
        if 0 < pos < len(chars):  # вставляем между символами
            chars.insert(pos, " ")
    return "".join(chars)


samples_shown = 0
for rec in iter_jsonl(VAL_PATH):
    text = rec["text_no_spaces"]
    gold = set(rec["boundaries"])
    probs = predict_probs_for_text(text)
    preds = set((np.where(probs >= best_thr)[0] + 1).tolist())
    s_gold = restore_with_positions(text, gold)
    s_pred = restore_with_positions(text, preds)
    print("no_spaces:", text[:120])
    print("GOLD     :", s_gold[:120])
    print("PRED     :", s_pred[:120])
    print("F1 =", f1_per_string(gold, preds))
    print("---")
    samples_shown += 1
    if samples_shown >= 5:
        break


Сохранено:
  Модель: /Users/strafe/VSCode-Projects/Avito-Test/artifacts_boundary/boundary_sgd.joblib
  Конфиг: /Users/strafe/VSCode-Projects/Avito-Test/artifacts_boundary/config_boundary.json
  Порог : /Users/strafe/VSCode-Projects/Avito-Test/artifacts_boundary/best_threshold.txt
no_spaces: Литва́(),официальноеназвание—Лито́вскаяРеспу́блика()—государство,расположенноевСевернойЕвропе.
GOLD     : Литва́ ( ), официальное название — Лито́вская Респу́блика ( ) — государство, расположенное в Северной Европе .
PRED     : Литва́ (), официальное на звание — Лито́вская Респу́блика () — государство, рас положенное в Северной Европе.
F1 = 0.8387096774193549
---
no_spaces: Так,например,существуютсозвучныетопонимынатерриторииСловакии(Lytva)иРумынии(Litua),известныесXI—XIIвеков.
GOLD     : Так, например, существуют созвучные топонимы на территории Словакии (Lytva) и Румынии (Litua), известные с XI—XII веков 
PRED     : Так, на пример, существ ую т созвучные топонимынатерритории Словакии (Lytva) и Рум

## Декодер

In [ ]:
from pathlib import Path
import json
import math
import numpy as np
from collections import Counter
from scipy import sparse
import joblib

# Пути
PREP_DIR = Path("prepared_corpus")
VAL_PATH = PREP_DIR / "val.jsonl"
UNIGRAM_PATH = PREP_DIR / "unigram_freq.tsv"

ART_DIR = Path("artifacts_boundary")
MODEL_PATH = ART_DIR / "boundary_sgd.joblib"
CFG_PATH = ART_DIR / "config_boundary.json"
THR_PATH = ART_DIR / "best_threshold.txt"  # пригодится для сравнения

# Проверки
assert MODEL_PATH.exists(), f"Нет модели: {MODEL_PATH}"
assert CFG_PATH.exists(),   f"Нет конфига: {CFG_PATH}"
assert VAL_PATH.exists(),   f"Нет валидации: {VAL_PATH}"
assert UNIGRAM_PATH.exists(), f"Нет униграмм: {UNIGRAM_PATH}"

# Загрузка
clf = joblib.load(MODEL_PATH)
cfg = json.loads(Path(CFG_PATH).read_text(encoding="utf-8"))
best_thr = float(Path(THR_PATH).read_text(
    encoding="utf-8").strip()) if THR_PATH.exists() else 0.5

# Важные гиперпараметры из Этапа 3
WINDOW = int(cfg["window"])
N_FEATURES = int(cfg["n_features"])
USE_UNIGRAM = bool(cfg["use_unigram"])
UNIGRAM_MIN_COUNT = int(cfg["unigram_min_count"] or 0)

print("Модель:", MODEL_PATH.name)
print("CFG   :", cfg)
print("best_thr из Этапа 3:", best_thr)


Модель: boundary_sgd.joblib
CFG   : {'window': 5, 'n_features': 262144, 'neg_downsample': 0.3, 'alpha': 1e-05, 'use_unigram': True, 'unigram_min_count': 5, 'seed': 42}
best_thr из Этапа 3: 0.525


In [ ]:
from sklearn.feature_extraction import FeatureHasher

# Хешер без обучаемых параметров — достаточно восстановить по N_FEATURES
HASher = FeatureHasher(n_features=N_FEATURES,
                       input_type="dict", alternate_sign=False)

# Если функции признаков уже объявлены в ноутбуке (Этап 3), используем их.
# Если нет — определим здесь «идентичные»:


def is_cyr(ch: str) -> bool:
    return 'А' <= ch <= 'я' or ch in "Ёё"


def is_lat(ch: str) -> bool:
    oc = ord(ch)
    return (65 <= oc <= 90) or (97 <= oc <= 122)


def is_digit(ch: str) -> bool:
    return ch.isdigit()


def is_punct(ch: str) -> bool:
    return ch in r""".,;:!?-—()[]{}«»"'/\|+*=_~%^<>…"""


def cat(ch: str) -> str:
    if is_cyr(ch):
        return "cyr"
    if is_lat(ch):
        return "lat"
    if is_digit(ch):
        return "dig"
    if is_punct(ch):
        return "pnc"
    return "oth"


def trans(a: str, b: str) -> str:
    return f"{a}->{b}"


def is_word_char(ch: str) -> bool:
    return is_cyr(ch) or is_lat(ch) or is_digit(ch) or ch in "-_"


def local_tokens(text: str, pos: int) -> tuple[str, str, str]:
    n = len(text)
    i = pos - 1
    j = pos
    L = i
    while L >= 0 and is_word_char(text[L]):
        L -= 1
    left_tok = text[L+1:i+1]
    R = j
    while R < n and is_word_char(text[R]):
        R += 1
    right_tok = text[j:R]
    merged = (left_tok + right_tok) if (left_tok and right_tok) else ""
    return left_tok.lower(), right_tok.lower(), merged.lower()


# LEXICON соберём на следующем шаге из unigram_freq.tsv
LEXICON = None


def boundary_features(text: str, pos: int, window: int, lexicon=None) -> dict:
    feats = {}
    n = len(text)
    Lch = text[pos-1]
    Rch = text[pos]
    Lc = cat(Lch)
    Rc = cat(Rch)

    feats[f"LC:{Lch}"] = 1
    feats[f"RC:{Rch}"] = 1
    feats[f"LCAT:{Lc}"] = 1
    feats[f"RCAT:{Rc}"] = 1
    feats[f"TRAN:{trans(Lc,Rc)}"] = 1

    feats["L_isupper"] = 1 if Lch.isalpha() and Lch.isupper() else 0
    feats["R_isupper"] = 1 if Rch.isalpha() and Rch.isupper() else 0

    for k in range(1, window+1):
        if pos-1-k >= 0:
            ch = text[pos-1-k]
            feats[f"L{k}:{ch}"] = 1
            feats[f"L{k}C:{cat(ch)}"] = 1
        if pos+k < n:
            ch = text[pos+k]
            feats[f"R{k}:{ch}"] = 1
            feats[f"R{k}C:{cat(ch)}"] = 1

    feats["dig_to_alpha"] = 1 if (
        is_digit(Lch) and (is_cyr(Rch) or is_lat(Rch))) else 0
    feats["alpha_to_dig"] = 1 if (
        (is_cyr(Lch) or is_lat(Lch)) and is_digit(Rch)) else 0
    feats["cyr_to_lat"] = 1 if (is_cyr(Lch) and is_lat(Rch)) else 0
    feats["lat_to_cyr"] = 1 if (is_lat(Lch) and is_cyr(Rch)) else 0
    feats["punct_left"] = 1 if is_punct(Lch) else 0
    feats["punct_right"] = 1 if is_punct(Rch) else 0

    if lexicon is not None:
        lt, rt, merged = local_tokens(text, pos)
        feats["lex_left"] = 1 if lt and lt in lexicon else 0
        feats["lex_right"] = 1 if rt and rt in lexicon else 0
        feats["lex_merged"] = 1 if merged and merged in lexicon else 0
        feats["len_lt"] = len(lt)
        feats["len_rt"] = len(rt)
        feats["len_merged"] = len(merged)
    else:
        feats["len_lt"] = 0
        feats["len_rt"] = 0
        feats["len_merged"] = 0

    feats["rel_pos"] = pos / n
    feats["is_start_near"] = 1 if pos <= 2 else 0
    feats["is_end_near"] = 1 if (n - pos) <= 2 else 0
    return feats


In [ ]:
class UnigramLM:
    def __init__(self, path: Path, min_count: int = 1, k: float = 0.5, lowercase: bool = True):
        """
        path: tsv word \t count
        k: add-k сглаживание (0.5 по умолчанию — хорошая середина)
        """
        self.k = float(k)
        self.lowercase = lowercase
        self.counts = {}
        total = 0
        vocab = 0

        with path.open("r", encoding="utf-8") as f:
            for line in f:
                w, c = line.rstrip("\n").split("\t")
                c = int(c)
                if c < min_count:
                    continue
                if lowercase:
                    w = w.lower()
                self.counts[w] = self.counts.get(w, 0) + c

        self.V = len(self.counts)
        self.N = sum(self.counts.values())

        # Предварительные лог-константы
        self._log_N_kV = math.log(self.N + self.k * max(self.V, 1))
        self._oov_log = math.log(self.k) - self._log_N_kV

    def logp(self, word: str) -> float:
        if not word:
            return self._oov_log
        w = word.lower() if self.lowercase else word
        cnt = self.counts.get(w, 0)
        if cnt == 0:
            return self._oov_log
        return math.log(cnt + self.k) - self._log_N_kV


# Загружаем лексикон: множество «известных слов» (флаги для признаков)
LEXICON = set()
with UNIGRAM_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        w, c = line.rstrip("\n").split("\t")
        if int(c) >= UNIGRAM_MIN_COUNT:
            LEXICON.add(w.lower())

LM = UnigramLM(UNIGRAM_PATH, min_count=1, k=0.5, lowercase=True)

print("UnigramLM: V=", LM.V, "N=", LM.N, "OOV_log≈", round(LM._oov_log, 3))
print("LEXICON size:", len(LEXICON))


UnigramLM: V= 1570271 N= 50360333 OOV_log≈ -18.443
LEXICON size: 385143


In [ ]:
# === FIX: DP со "средним за символ" языком и бонусом словаря ===
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def boundary_probs(text: str) -> np.ndarray:
    n = len(text)
    if n <= 1:
        return np.zeros((0,), dtype=np.float32)
    feats = [boundary_features(text, pos, WINDOW, LEXICON)
             for pos in range(1, n)]
    X = HASher.transform(feats)
    dec = clf.decision_function(X)
    return sigmoid(dec).astype(np.float32)


def safe_logit(p: float, eps=1e-6) -> float:
    p = min(max(p, eps), 1.0 - eps)
    return math.log(p) - math.log(1.0 - p)


def boundary_logits_from_probs(pvec: np.ndarray, eps=1e-6) -> np.ndarray:
    return np.array([safe_logit(float(p), eps) for p in pvec], dtype=np.float32)


# ТЮНИНГ (стартовые числа, подправь по val):
ALPHA = 3.0    # вес логитов классификатора границ
BETA = 0.20   # вес языкового слагаемого (за символ)
GAMMA = 0.02   # регуляризация длины
TAU = 0.30   # бонус, если слово в лексиконе (LEXICON)
# константный bias за каждую поставленную границу (можно 0.1..0.4 при недосегментации)
DELTA = 0.00
L_MAX = 20     # макс. длина слова


def length_reg(len_word: int) -> float:
    pen = 0.0
    if len_word <= 2:
        pen -= 0.8
    if len_word >= 25:
        pen -= (len_word - 24) * 0.15
    return pen


def dp_segment(text: str,
               alpha: float = ALPHA, beta: float = BETA, gamma: float = GAMMA,
               tau: float = TAU, delta: float = DELTA, L: int = L_MAX):
    n = len(text)
    if n <= 1:
        return set(), 0.0

    p = boundary_probs(text)                  # shape (n-1,)
    logit = boundary_logits_from_probs(p)     # shape (n-1,)

    dp = np.full(n + 1, -1e15, dtype=np.float64)
    bp = np.full(n + 1, -1, dtype=np.int32)
    dp[0] = 0.0

    for i in range(1, n + 1):
        j_min = max(0, i - L)
        best_local, best_j = -1e15, -1
        for j in range(i - 1, j_min - 1, -1):
            w = text[j:i]
            # Язык: средний лог-проб за символ (не копит штраф за число слов)
            lp_per_char = LM.logp(w) / max(1, len(w))
            # Бонус за словарность
            lex_bonus = tau if (w.lower() in LEXICON) else 0.0
            # Регуляризация длины
            reg = length_reg(len(w))
            # Скор границы перед словом (если не начало): логит + bias DELTA
            bscore = (alpha * logit[j - 1] + delta) if j > 0 else 0.0

            cur = dp[j] + beta * lp_per_char + lex_bonus + gamma * reg + bscore
            if cur > best_local:
                best_local, best_j = cur, j
        dp[i], bp[i] = best_local, best_j

    # backtrack → позиции пробелов
    pos = set()
    i = n
    while i > 0:
        j = int(bp[i])
        if j < 0:
            break
        if j > 0:
            pos.add(j)
        i = j
    return pos, float(dp[n])


In [ ]:
def f1_per_string(gold_set, pred_set):
    if not gold_set and not pred_set:
        return 1.0
    if not gold_set or not pred_set:
        return 0.0
    tp = len(gold_set & pred_set)
    fp = len(pred_set - gold_set)
    fn = len(gold_set - pred_set)
    if tp == 0:
        return 0.0
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    return 2 * prec * rec / (prec + rec)


# Оценка DP на валидации
f1s = []
cnt = 0
for line in Path(VAL_PATH).open("r", encoding="utf-8"):
    if not line.strip():
        continue
    rec = json.loads(line)
    text = rec["text_no_spaces"]
    gold = set(rec["boundaries"])
    pred, _ = dp_segment(text, alpha=ALPHA, beta=BETA, gamma=GAMMA, L=L_MAX)
    f1s.append(f1_per_string(gold, pred))
    cnt += 1
    if cnt % 2000 == 0:
        print(
            f"[val:DP] обработано {cnt} строк, текущий F1={np.mean(f1s)*100:.2f}")

mean_f1_dp = float(np.mean(f1s)) if f1s else 0.0
print(f"\n[val:DP] Средний F1 по позициям: {mean_f1_dp*100:.2f}")


[val:DP] обработано 2000 строк, текущий F1=76.14
[val:DP] обработано 4000 строк, текущий F1=76.93
[val:DP] обработано 6000 строк, текущий F1=77.44
[val:DP] обработано 8000 строк, текущий F1=77.82
[val:DP] обработано 10000 строк, текущий F1=78.18
[val:DP] обработано 12000 строк, текущий F1=77.83
[val:DP] обработано 14000 строк, текущий F1=77.84
[val:DP] обработано 16000 строк, текущий F1=77.69
[val:DP] обработано 18000 строк, текущий F1=78.34
[val:DP] обработано 20000 строк, текущий F1=79.11
[val:DP] обработано 22000 строк, текущий F1=79.29
[val:DP] обработано 24000 строк, текущий F1=79.22
[val:DP] обработано 26000 строк, текущий F1=78.92
[val:DP] обработано 28000 строк, текущий F1=78.74
[val:DP] обработано 30000 строк, текущий F1=78.52
[val:DP] обработано 32000 строк, текущий F1=78.34
[val:DP] обработано 34000 строк, текущий F1=78.19
[val:DP] обработано 36000 строк, текущий F1=78.05
[val:DP] обработано 38000 строк, текущий F1=78.01
[val:DP] обработано 40000 строк, текущий F1=77.90
[val

In [ ]:
# Жадный порог: ставим пробел там, где p >= best_thr (для сравнения)
def greedy_positions(text: str, thr: float) -> set[int]:
    p = boundary_probs(text)
    idx = (np.where(p >= thr)[0] + 1).tolist()  # p[0] -> позиция 1
    return set(idx)


f1s_greedy = []
cnt = 0
for line in Path(VAL_PATH).open("r", encoding="utf-8"):
    if not line.strip():
        continue
    rec = json.loads(line)
    text = rec["text_no_spaces"]
    gold = set(rec["boundaries"])
    pred = greedy_positions(text, best_thr)
    f1s_greedy.append(f1_per_string(gold, pred))
    cnt += 1
    if cnt % 2000 == 0:
        print(
            f"[val:Greedy] обработано {cnt} строк, текущий F1={np.mean(f1s_greedy)*100:.2f}")

mean_f1_greedy = float(np.mean(f1s_greedy)) if f1s_greedy else 0.0
print(f"\n[val:Greedy] Средний F1 по позициям: {mean_f1_greedy*100:.2f}")
print(
    f"[сравнение] DP - Greedy: {(mean_f1_dp-mean_f1_greedy)*100:.2f} пунктов")


[val:Greedy] обработано 2000 строк, текущий F1=74.05
[val:Greedy] обработано 4000 строк, текущий F1=74.94
[val:Greedy] обработано 6000 строк, текущий F1=75.44
[val:Greedy] обработано 8000 строк, текущий F1=75.87
[val:Greedy] обработано 10000 строк, текущий F1=76.23
[val:Greedy] обработано 12000 строк, текущий F1=75.91
[val:Greedy] обработано 14000 строк, текущий F1=75.95
[val:Greedy] обработано 16000 строк, текущий F1=75.83
[val:Greedy] обработано 18000 строк, текущий F1=76.52
[val:Greedy] обработано 20000 строк, текущий F1=77.29
[val:Greedy] обработано 22000 строк, текущий F1=77.49
[val:Greedy] обработано 24000 строк, текущий F1=77.46
[val:Greedy] обработано 26000 строк, текущий F1=77.15
[val:Greedy] обработано 28000 строк, текущий F1=76.95
[val:Greedy] обработано 30000 строк, текущий F1=76.75
[val:Greedy] обработано 32000 строк, текущий F1=76.57
[val:Greedy] обработано 34000 строк, текущий F1=76.43
[val:Greedy] обработано 36000 строк, текущий F1=76.29
[val:Greedy] обработано 38000 ст

In [ ]:
def restore_with_positions(text_no_spaces: str, positions: set[int]) -> str:
    chars = list(text_no_spaces)
    for pos in sorted(positions, reverse=True):
        if 0 <= pos <= len(chars):
            chars.insert(pos, " ")
    return "".join(chars)


shown = 0
for line in Path(VAL_PATH).open("r", encoding="utf-8"):
    rec = json.loads(line)
    text = rec["text_no_spaces"]
    gold = set(rec["boundaries"])
    pred, _ = dp_segment(text, alpha=ALPHA, beta=BETA, gamma=GAMMA, L=L_MAX)

    print("no_spaces:", text[:140])
    print("GOLD     :", restore_with_positions(text, gold)[:140])
    print("DP       :", restore_with_positions(text, pred)[:140])
    print("F1=", f1_per_string(gold, pred))
    print("---")
    shown += 1
    if shown >= 5:
        break


no_spaces: Литва́(),официальноеназвание—Лито́вскаяРеспу́блика()—государство,расположенноевСевернойЕвропе.
GOLD     : Литва́ ( ), официальное название — Лито́вская Респу́блика ( ) — государство, расположенное в Северной Европе .
DP       : Литва́ (), официальное на звание — Лито́вская Респу́блика () — государство, рас положенное в Северной Европе.
F1= 0.8387096774193549
---
no_spaces: Так,например,существуютсозвучныетопонимынатерриторииСловакии(Lytva)иРумынии(Litua),известныесXI—XIIвеков.
GOLD     : Так, например, существуют созвучные топонимы на территории Словакии (Lytva) и Румынии (Litua), известные с XI—XII веков .
DP       : Так, на пример, существ ую т созвучные топонимынатерритории Словакии (Lytva) и Румынии (Litua), известные с XI— XII веков.
F1= 0.787878787878788
---
no_spaces: Феодальноекняжество,поземлямкоторогопротекалаэтарека,современемзаняловедущееположениеиназваниебылораспространенонавсёгосударство.
GOLD     : Феодальное княжество, по землям которого протекала эта река, с

## Эвристики

In [ ]:
# === ЭТАП 5. ЭВРИСТИКИ ВЫСОКОЙ ТОЧНОСТИ ДЛЯ ДОКРУТКИ ПОСЛЕ DP ===
# Требует: dp_segment(text) и LEXICON/LM/HASher/фичи уже определены (из Этапов 3–4).

import re
from typing import Set, Tuple, List

# 1) Набор единиц измерения (ваш "первый вариант" — без расширений)
UNITS = ["см", "мм", "м", "км", "кг", "г", "л", "мл", "дюйм", "дюйма", "дюймов",
         "gb", "mb", "tb", "гб", "мб", "тб"]
UNITS_RE = r"(?:%s)\b" % "|".join(map(re.escape, UNITS))

# 2) Белые списки «не делить внутри» (бренд+модель, GPU и т.п.)
_WHITELIST_PATTERNS = [
    r"(?:iphone|samsung|xiaomi|huawei|sony|nokia|poco|asus|lenovo|honor|pixel|oneplus)\d+[a-z]*",
    r"(?:rtx|gtx|rx)\d{3,4}[a-z]*",
    r"(?:arc)\d+",
    r"(?:s|mi|k)\d{1,3}[a-z]*",   # s23, mi14, k50, 12t и т.п.
]
WHITELIST = [re.compile(p, flags=re.IGNORECASE) for p in _WHITELIST_PATTERNS]

# 3) Пунктуация, где пробел «почти всегда уместен» ПОСЛЕ символа
# точку сознательно НЕ включаем (числа вида 2.3кг)
AFTER_PUNCT = set(",:;!?)]»")
# 4) Открывающие скобки/кавычки — пробел обычно НУЖЕН ПЕРЕД ними
BEFORE_OPEN = set("([«")


def _collect_whitelist_spans(text: str) -> List[Tuple[int, int]]:
    spans = []
    for rgx in WHITELIST:
        for m in rgx.finditer(text):
            spans.append(m.span())
    return spans


def _pos_in_spans(pos: int, spans: List[Tuple[int, int]]) -> bool:
    # Внутри матчинга (не на краю) — не ставим пробелы
    # span = [L, R), pos — разрез между pos-1 и pos
    for L, R in spans:
        if L < pos < R:
            return True
    return False


def _add_boundary_safe(n: int, pos: int, positions: Set[int]) -> None:
    if 0 < pos < n:
        positions.add(pos)


def apply_heuristics(text: str, base_positions: Set[int]) -> Set[int]:
    """
    text: компактная строка без пробелов
    base_positions: множество позиций (0-based), где DP решил ставить пробел
    Возвращает обновлённое множество позиций с учётом эвристик.
    """
    n = len(text)
    if n <= 1:
        return set()

    positions = set(base_positions)  # копия
    spans = _collect_whitelist_spans(text)

    # H1) Число + единица измерения: 27дюймов -> 27 дюймов, 512gb -> 512 gb
    # Ищем ...<digits>(<unit>)
    for m in re.finditer(r"(\d+)" + UNITS_RE, text, flags=re.IGNORECASE):
        unit_start = m.start(
            2) if m.lastindex and m.lastindex >= 2 else m.end(1)
        # Разрез ставим ПЕРЕД unit
        if not _pos_in_spans(unit_start, spans):
            _add_boundary_safe(n, unit_start, positions)

    # H2) Буква↔цифра и цифра↔буква — ставим пробел, если НЕ попадаем в белый список
    for pos in range(1, n):
        L, R = text[pos-1], text[pos]
        is_alpha_L = L.isalpha()
        is_alpha_R = R.isalpha()
        is_dig_L = L.isdigit()
        is_dig_R = R.isdigit()
        if (is_alpha_L and is_dig_R) or (is_dig_L and is_alpha_R):
            if not _pos_in_spans(pos, spans):
                _add_boundary_safe(n, pos, positions)

    # H3) Пунктуация: пробел ПОСЛЕ , : ; ! ? ) ] »
    for pos in range(1, n):
        L, R = text[pos-1], text[pos]
        if L in AFTER_PUNCT and (R.isalpha() or R.isdigit()):
            _add_boundary_safe(n, pos, positions)

    # H4) Открывающие скобки/кавычки: пробел ПЕРЕД ( [ «
    for pos in range(1, n):
        L, R = text[pos-1], text[pos]
        if R in BEFORE_OPEN and (L.isalpha() or L.isdigit()):
            _add_boundary_safe(n, pos, positions)

    # H5) Мини-санити: не даём пробел в начале/конце (и удалим очевидный мусор)
    positions = {p for p in positions if 0 < p < n}

    return positions

# --- Быстрая проверка эффекта эвристик на валидации ---


def f1_per_string(gold_set, pred_set):
    if not gold_set and not pred_set:
        return 1.0
    if not gold_set or not pred_set:
        return 0.0
    tp = len(gold_set & pred_set)
    fp = len(pred_set - gold_set)
    fn = len(gold_set - pred_set)
    if tp == 0:
        return 0.0
    prec = tp / (tp + fp)
    rec = tp / (tp + fn)
    return 2 * prec * rec / (prec + rec)


val_path = Path("prepared_corpus/val.jsonl")
f1s_dp, f1s_dp_h = [], []
checked = 0
for line in val_path.open("r", encoding="utf-8"):
    rec = json.loads(line)
    t = rec["text_no_spaces"]
    gold = set(rec["boundaries"])
    pos_dp, _ = dp_segment(t)                 # DP без эвристик
    pos_h = apply_heuristics(t, pos_dp)       # DP + эвристики
    f1s_dp.append(f1_per_string(gold, pos_dp))
    f1s_dp_h.append(f1_per_string(gold, pos_h))
    checked += 1
    if checked % 2000 == 0:
        print(
            f"[val:heur] {checked} строк — DP={np.mean(f1s_dp)*100:.2f}, DP+H={np.mean(f1s_dp_h)*100:.2f}")

print(
    f"\n[val:heur] Итог по {checked} строкам — DP={np.mean(f1s_dp)*100:.2f}, DP+H={np.mean(f1s_dp_h)*100:.2f}")


[val:heur] 2000 строк — DP=76.14, DP+H=76.15
[val:heur] 4000 строк — DP=76.93, DP+H=77.03
[val:heur] 6000 строк — DP=77.44, DP+H=77.58
[val:heur] 8000 строк — DP=77.82, DP+H=77.98
[val:heur] 10000 строк — DP=78.18, DP+H=78.33
[val:heur] 12000 строк — DP=77.83, DP+H=77.98
[val:heur] 14000 строк — DP=77.84, DP+H=77.99
[val:heur] 16000 строк — DP=77.69, DP+H=77.84
[val:heur] 18000 строк — DP=78.34, DP+H=78.50
[val:heur] 20000 строк — DP=79.11, DP+H=79.27
[val:heur] 22000 строк — DP=79.29, DP+H=79.45
[val:heur] 24000 строк — DP=79.22, DP+H=79.38
[val:heur] 26000 строк — DP=78.92, DP+H=79.08
[val:heur] 28000 строк — DP=78.74, DP+H=78.88
[val:heur] 30000 строк — DP=78.52, DP+H=78.69
[val:heur] 32000 строк — DP=78.34, DP+H=78.50
[val:heur] 34000 строк — DP=78.19, DP+H=78.36
[val:heur] 36000 строк — DP=78.05, DP+H=78.23
[val:heur] 38000 строк — DP=78.01, DP+H=78.19
[val:heur] 40000 строк — DP=77.90, DP+H=78.07
[val:heur] 42000 строк — DP=77.83, DP+H=78.00
[val:heur] 44000 строк — DP=77.81, DP+

## Submit

In [ ]:
# === Сабмит в формате: id, predicted_positions (JSON-строка вида "[5, 8, 13]") ===
import json
import pandas as pd
from pathlib import Path

INPUT_PATH = Path("dataset_1937770_3.txt")      # твой файл для инференса
OUTPUT_PATH = Path("submission.csv")

# берём лоадер из ранних ячеек; если его нет — вот минимальная версия:


def load_task_data_txt(path: Path) -> pd.DataFrame:
    raw = path.read_text(encoding="utf-8-sig").splitlines()
    assert raw and raw[0].strip(
    ) == "id,text_no_spaces", "Ожидается заголовок 'id,text_no_spaces'"
    rows = []
    for i, line in enumerate(raw[1:], start=2):
        if not line:
            continue
        left, right = line.split(",", 1)
        rows.append((left, right))
    df = pd.DataFrame(rows, columns=["id", "text_no_spaces"])
    df["id"] = df["id"].astype(str)
    df["text_no_spaces"] = df["text_no_spaces"].astype(str)
    return df

# форматирование позиций: JSON с пробелами (как в примере)


def format_positions_json(pos_set) -> str:
    arr = sorted(int(p) for p in pos_set)
    # даёт "[1, 4, 9]" со пробелами после запятой
    return json.dumps(arr, ensure_ascii=False)


# --- инференс для всего файла ---
task = load_task_data_txt(INPUT_PATH)

pred_col = []
for i, text in enumerate(task["text_no_spaces"].tolist(), 1):
    # твоя функция, которая возвращает множество позиций:
    # pos = infer_positions_for_row(text)  # если уже определена
    # Если она не определена в этом ноутбуке, подключи из Этапа 4–5:
    pos_dp, _ = dp_segment(text)                 # DP
    pos = apply_heuristics(text, pos_dp)         # эвристики поверх DP

    pred_col.append(format_positions_json(pos))
    if i % 500 == 0:
        print(f"[infer] обработано: {i}")

submission = task[["id", "text_no_spaces"]].copy()
submission["predicted_positions"] = pred_col

submission.to_csv(OUTPUT_PATH, index=False)
print("Сохранено:", OUTPUT_PATH.resolve())

# Быстрый просмотр
display(submission.head(5))
print(submission.dtypes)


[infer] обработано: 500
[infer] обработано: 1000
Сохранено: /Users/strafe/VSCode-Projects/Avito-Test/submission.csv


,id,text_no_spaces,predicted_positions
0,0,куплюайфон14про,"[7, 10, 12]"
1,1,ищудомвПодмосковье,"[6, 7]"
2,2,сдаюквартирусмебельюитехникой,"[5, 21]"
3,3,новыйдивандоставканедорого,"[5, 16, 18, 20]"
4,4,отдамдаромкошку,"[2, 5, 10]"


id                     object
text_no_spaces         object
predicted_positions    object
dtype: object
